<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Pre-Process,-Training,-and-Modeling" data-toc-modified-id="Pre-Process,-Training,-and-Modeling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-Process, Training, and Modeling</a></span><ul class="toc-item"><li><span><a href="#Pre-processing-of-well-log-analysis" data-toc-modified-id="Pre-processing-of-well-log-analysis-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Pre-processing of well log analysis</a></span><ul class="toc-item"><li><span><a href="#Classification" data-toc-modified-id="Classification-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Classification</a></span></li></ul></li></ul></li><li><span><a href="#Pycaret" data-toc-modified-id="Pycaret-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pycaret</a></span></li></ul></div>

# Pre-Process, Training, and Modeling

## Pre-processing of well log analysis

Using the notebook with the well log, I interpreted the well logs in an excel sheet (using Ko Ko Rules in petrophysical interpretation), looking at Gamma ray, resistance, and Density and Neutron Porosities. 

In [3]:
!pip install numpy

  Using cached numpy-1.19.5-cp36-cp36m-win_amd64.whl (13.2 MB)


In [3]:
!pip install pyexcel-xlsx

In [6]:
!pip install xlrd==1.2.0

  Attempting uninstall: xlrd
    Found existing installation: xlrd 2.0.1
    Uninstalling xlrd-2.0.1:
      Successfully uninstalled xlrd-2.0.1


In [1]:
!pip install XlsxWriter

In [2]:
!pip install xlwings

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for xlwings: filename=xlwings-0.25.0-py3-none-any.whl size=818556 sha256=c5ff667c2ac5fb1ab7c5f6175b17e56520f6ab00dca0b3b40624c453999f6ae0
  Stored in directory: c:\users\yuvem\appdata\local\pip\cache\wheels\9c\09\09\c3d1e62eee88d3b10115fad734138948020c21a1927b9feb5d
Successfully built xlwings


In [3]:
!pip install xlwt

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# Read file
path0 = r'C:\Users\yuvem\OneDrive\Documents\table_well_log.xlsx'
table = pd.read_excel(path0)
table.head()

,well_id,gamma_ray,resistivity,bulk_density,neutron_porosity,neutron_porosity_density_crossover,hydrocarbon_present
0,0,left,right,left,left,yes,yes
1,1,left,left,right,left,yes,yes
2,2,left,right,left,right,no,no
3,3,left,right,left,left,yes,yes
4,4,left,right,left,left,yes,yes


In [3]:
df = table.set_index('well_id')

In [4]:
df.isnull().sum()

gamma_ray                             4
resistivity                           1
bulk_density                          4
neutron_porosity                      4
neutron_porosity_density_crossover    4
hydrocarbon_present                   4
dtype: int64

In [5]:
# Clear nulls
df = df.dropna()
df.isnull().sum()
print(len(df))

32


In [6]:
df.resistivity.unique()

array(['right', 'left'], dtype=object)

### Classification

In [7]:
# Get x and y parameters
x = df.iloc[: , :-1]
feat =(['gamma_ray','resistivity','bulk_density','neutron_porosity','neutron_porosity_density_crossover'])
y = df.iloc[:,5].values

In [8]:
label_x = LabelEncoder()
x = x.apply(LabelEncoder().fit_transform)

In [9]:
rfc = RandomForestClassifier(max_depth=None,
                             criterion='gini')
print(x,feat)

         gamma_ray   resistivity  bulk_density  neutron_porosity  \
well_id                                                            
0                 1            1             0                 0   
1                 1            0             1                 0   
2                 1            1             0                 1   
3                 1            1             0                 0   
4                 1            1             0                 0   
5                 1            1             0                 0   
7                 1            1             1                 1   
8                 1            1             1                 0   
9                 1            1             0                 0   
10                1            1             1                 0   
11                1            1             0                 1   
12                1            1             0                 1   
13                1            1             0  

In [10]:
# Train
rfc.fit(x, y)

RandomForestClassifier()

In [11]:
#prediction
x_in = np.array([1,1,0,0,1])
x_in
y_pred = rfc.predict([x_in])
print("presence of hydrocarbon at this depth=",y_pred)

presence of hydrocarbon at this depth= ['yes']


In [12]:
# Accuracy
rfc.score(x,y)*100

96.875

In [13]:
# Decision Tree classifier
dtc = DecisionTreeClassifier(criterion='gini',max_depth=None,splitter='best')
dtc.fit(x, y)

DecisionTreeClassifier()

In [14]:
# Prediction for Decision Tree Classifier
x_in2 = np.array([1,1,0,0,1])
x_in2 
y_pred2 = dtc.predict([x_in2])
print("presence of hydrocarbon at this depth=",y_pred2)

presence of hydrocarbon at this depth= ['yes']


In [15]:
dtc.score(x, y)*100

96.875

# Pycaret

In [ ]:
!pip install pycaret

In [16]:
import pycaret
from pycaret.regression import *

In [18]:
s = setup(df, target = 'resistivity', session_id = 123)

,Description,Value
0,session_id,123
1,Target,resistivity
2,Original Data,"(32, 6)"
3,Missing Values,False
4,Numeric Features,0
5,Categorical Features,5
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(22, 5)"


In [19]:
# check transformed X_train
get_config('X_train')

,gamma_ray _left,bulk_density_left,neutron_porosity_left,neutron_porosity_density_crossover_no,hydrocarbon_present_no
well_id,,,,,
21,1.0,1.0,1.0,0.0,0.0
4,1.0,1.0,1.0,0.0,0.0
3,1.0,1.0,1.0,0.0,0.0
28,1.0,1.0,0.0,0.0,1.0
19,1.0,1.0,1.0,0.0,1.0
16,1.0,1.0,0.0,0.0,1.0
23,1.0,1.0,1.0,0.0,0.0
10,1.0,0.0,1.0,0.0,1.0
17,1.0,1.0,1.0,0.0,0.0


In [20]:
# train all models using default hyperparameters
best = compare_models()

In [21]:
print(best)

[]
